In [33]:
import pandas as pd
import yfinance as yf
import ta

In [ ]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

sp500["Symbol"] = sp500["Symbol"].str.replace('.', "-")

In [ ]:
sp500

In [ ]:
symbol_list = sp500["Symbol"].unique().tolist()

In [ ]:
end_date = "2024-02-25"

start_date = "2015-01-30"

df = yf.download(tickers=symbol_list,
                 start=start_date,
                 end=end_date).stack()

In [ ]:
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()  
df

In [ ]:
df.to_csv("stock.csv")

In [3]:
stocks = pd.read_csv("Data/stock.csv")
stocks.tail()

,date,ticker,adj close,close,high,low,open,volume
1120118,2024-02-23,XYL,125.629997,125.629997,126.540001,125.370003,125.660004,739200.0
1120119,2024-02-23,YUM,137.643600,138.320007,138.669998,137.100006,137.580002,1592200.0
1120120,2024-02-23,ZBH,128.937531,129.179993,129.509995,128.080002,128.630005,1023500.0
1120121,2024-02-23,ZBRA,274.989990,274.989990,279.829987,274.989990,278.209991,269300.0
1120122,2024-02-23,ZTS,196.649292,197.210007,198.210007,194.820007,197.350006,2391100.0


In [4]:
stocks

,date,ticker,adj close,close,high,low,open,volume
0,2015-01-30,A,34.968155,37.770000,38.320000,37.709999,38.009998,3054300.0
1,2015-01-30,AAL,46.503475,49.080002,52.130001,48.630001,52.020000,18473100.0
2,2015-01-30,AAPL,26.149805,29.290001,30.000000,29.212500,29.600000,334982000.0
3,2015-01-30,ABBV,40.893211,60.349998,62.680000,60.119999,62.090000,17582500.0
4,2015-01-30,ABT,37.562355,44.759998,45.630001,44.700001,45.400002,9613300.0
...,...,...,...,...,...,...,...,...
1120118,2024-02-23,XYL,125.629997,125.629997,126.540001,125.370003,125.660004,739200.0
1120119,2024-02-23,YUM,137.643600,138.320007,138.669998,137.100006,137.580002,1592200.0
1120120,2024-02-23,ZBH,128.937531,129.179993,129.509995,128.080002,128.630005,1023500.0
1120121,2024-02-23,ZBRA,274.989990,274.989990,279.829987,274.989990,278.209991,269300.0


In [5]:
tickers = stocks['ticker'].unique().tolist()
tickers = sorted(tickers)

In [6]:


# Set the index to be a MultiIndex with 'ticker' and 'date'
multi_index_df = stocks.set_index(['ticker', 'date'])

multi_index_df.sort_index(inplace=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  
pd.set_option('display.max_rows', 100) 


multi_index_df


adj close       close        high         low        open     volume
ticker date                                                                             
A      2015-01-30   34.968155   37.770000   38.320000   37.709999   38.009998  3054300.0
       2015-02-02   35.819912   38.689999   38.759998   37.709999   37.900002  2508500.0
       2015-02-03   36.680912   39.619999   39.630001   38.919998   39.040001  2950400.0
       2015-02-04   36.208744   39.110001   39.459999   38.820000   39.330002  2567400.0
       2015-02-05   36.597603   39.529999   39.820000   39.220001   39.220001  1826300.0
...                       ...         ...         ...         ...         ...        ...
ZTS    2024-02-16  187.854370  188.389999  190.350006  187.929993  189.399994  1953700.0
       2024-02-20  186.019608  186.550003  189.410004  186.240005  187.300003  2502800.0
       2024-02-21  187.844406  188.380005  188.860001  186.660004  186.660004  3179300.0
       2024-02-22  196.100861  196.660004  196.669998  188.539993  189.309998  3339700.0
       2024-02-23  196.649292  197.210007  198.210007  194.820007  197.350006  2391100.0

[1120123 rows x 6 columns]

In [7]:
tickers

['A',
 'AAL',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ATO',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXON',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BDX',
 'BEN',
 'BF-B',
 'BG',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLDR',
 'BLK',
 'BMY',
 'BR',
 'BRK-B',
 'BRO',
 'BSX',
 'BWA',
 'BX',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COR',
 'COST',
 'CPAY',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTL

In [8]:
#Price
price = []
price.append((multi_index_df['high'] + multi_index_df['low'] + multi_index_df['adj close']) / 3)


#VWAP
pv = multi_index_df['volume'] * price[0]

cumu_pv = pv.cumsum()
cumu_vol = multi_index_df['volume'].cumsum()

multi_index_df['vwap'] = cumu_pv / cumu_vol
multi_index_df

adj close       close        high         low        open     volume       vwap
ticker date                                                                                        
A      2015-01-30   34.968155   37.770000   38.320000   37.709999   38.009998  3054300.0  36.999385
       2015-02-02   35.819912   38.689999   38.759998   37.709999   37.900002  2508500.0  37.193554
       2015-02-03   36.680912   39.619999   39.630001   38.919998   39.040001  2950400.0  37.615240
       2015-02-04   36.208744   39.110001   39.459999   38.820000   39.330002  2567400.0  37.742137
       2015-02-05   36.597603   39.529999   39.820000   39.220001   39.220001  1826300.0  37.855863
...                       ...         ...         ...         ...         ...        ...        ...
ZTS    2024-02-16  187.854370  188.389999  190.350006  187.929993  189.399994  1953700.0  71.439823
       2024-02-20  186.019608  186.550003  189.410004  186.240005  187.300003  2502800.0  71.439873
       2024-02-21  187.844406  188.380005  188.860001  186.660004  186.660004  3179300.0  71.439937
       2024-02-22  196.100861  196.660004  196.669998  188.539993  189.309998  3339700.0  71.440008
       2024-02-23  196.649292  197.210007  198.210007  194.820007  197.350006  2391100.0  71.440060

[1120123 rows x 7 columns]

In [9]:
multi_index_df

adj close       close        high         low        open     volume       vwap
ticker date                                                                                        
A      2015-01-30   34.968155   37.770000   38.320000   37.709999   38.009998  3054300.0  36.999385
       2015-02-02   35.819912   38.689999   38.759998   37.709999   37.900002  2508500.0  37.193554
       2015-02-03   36.680912   39.619999   39.630001   38.919998   39.040001  2950400.0  37.615240
       2015-02-04   36.208744   39.110001   39.459999   38.820000   39.330002  2567400.0  37.742137
       2015-02-05   36.597603   39.529999   39.820000   39.220001   39.220001  1826300.0  37.855863
...                       ...         ...         ...         ...         ...        ...        ...
ZTS    2024-02-16  187.854370  188.389999  190.350006  187.929993  189.399994  1953700.0  71.439823
       2024-02-20  186.019608  186.550003  189.410004  186.240005  187.300003  2502800.0  71.439873
       2024-02-21  187.844406  188.380005  188.860001  186.660004  186.660004  3179300.0  71.439937
       2024-02-22  196.100861  196.660004  196.669998  188.539993  189.309998  3339700.0  71.440008
       2024-02-23  196.649292  197.210007  198.210007  194.820007  197.350006  2391100.0  71.440060

[1120123 rows x 7 columns]

In [10]:
df = multi_index_df.copy()

In [11]:

processed_df = pd.DataFrame()

for tick in tickers:
    ticker_data = df.loc[tick].copy()
    
    # Calculate Relative Strength Index
    ticker_data['rsi'] = ta.momentum.rsi(ticker_data['close'], window=14)

    # Calculate Bollinger Bands
    bollinger = ta.volatility.BollingerBands(close=ticker_data['close'], window=20, window_dev=2)
    ticker_data['bb_low'] = bollinger.bollinger_lband()
    ticker_data['bb_mid'] = bollinger.bollinger_mavg()
    ticker_data['bb_high'] = bollinger.bollinger_hband()

    # Calculate Average True Range
    ticker_data['atr'] = ta.volatility.average_true_range(ticker_data['high'], ticker_data['low'], ticker_data['close'], window=14)

    # Calculate Moving Average Convergence Divergence
    macd = ta.trend.MACD(ticker_data['close'])
    ticker_data['macd'] = macd.macd()
    ticker_data['macd_signal'] = macd.macd_signal()
    
    # Calculate Moving Averages
    ticker_data['short_ma'] = ta.trend.sma_indicator(ticker_data['close'], window=20)  
    ticker_data['long_ma'] = ta.trend.sma_indicator(ticker_data['close'], window=50)
    
    ticker_data['return'] = ticker_data['adj close'].pct_change()
    
    ticker_data['rolling_mean'] = ticker_data['adj close'].rolling(window=20).mean()
    ticker_data['rolling_std'] = ticker_data['adj close'].rolling(window=20).std()
    
    ticker_data['lag1'] = ticker_data['adj close'].shift(1)
    ticker_data['lag2'] = ticker_data['adj close'].shift(2)
    ticker_data['lag3'] = ticker_data['adj close'].shift(3)
    
    ticker_data['return'] = ticker_data['adj close'].pct_change()

    
    ticker_data['target'] = (ticker_data['close'].shift(-1) > ticker_data['close']).astype(int)
    # Remove NaN values
    ticker_data = ticker_data.dropna(subset=['rsi', 'bb_low', 'bb_mid', 'bb_high', 'atr', 'macd', 'macd_signal', 'short_ma', 'long_ma', 'lag1', 'lag2', 'lag3', 'rolling_mean', 'rolling_std', 'return'])

    ticker_data['ticker'] = tick
    ticker_data = ticker_data.set_index('ticker', append=True)
    ticker_data = ticker_data.reorder_levels(['ticker', 'date'])


    processed_df = pd.concat([processed_df, ticker_data])
    
    



In [12]:
ticker = ['AAPL', 'A', 'AAL']

def plot_market(data, ticker:list, market:str):
    fig = go.Figure()
    for tick in ticker:
        if tick not in tickers:
            return f'{tick} not valid ticker'
    if market == 'volume':
        yaxis = 'Volume' 
    elif market in data.columns:
        yaxis = f'{market.capitalize()} Prince'
    else:
        return 'Enter valid Market'
        
    for tick in ticker:
        fig.add_trace(go.Scatter(
            x=data.loc[tick].index,
            y=data.loc[tick][market],
            mode='lines',
            name=tick
        ))
    fig.update_layout(
        title=f'Stock {market} for Multiple Ticker/s',
        xaxis_title='Date',            
        yaxis_title=yaxis,
        legend_title='Ticker'
    )
    fig.show()
    

plot_market(processed_df, ticker=ticker, market='short_ma' )


In [13]:
df = processed_df.copy()


In [14]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go



app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value=df.index.get_level_values(0).unique()[0]
    ),
    dcc.Graph(id='price-chart')
])

@app.callback(
    Output('price-chart', 'figure'),
    [Input('stock-selector', 'value')]
)
def update_chart(selected_stock):
    ticker_df = df.loc[selected_stock]
    figure = {
        'data': [
            go.Scatter(x=ticker_df.index, y=ticker_df['close'], mode='lines', name='Close Price'),
            go.Scatter(x=ticker_df.index, y=ticker_df['short_ma'], mode='lines', name='Short MA'),
            go.Scatter(x=ticker_df.index, y=ticker_df['long_ma'], mode='lines', name='Long MA')
        ],
        'layout': go.Layout(
            title=f'Stock Prices for {selected_stock}',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'}
        )
    }
    return figure

if __name__ == '__main__':
    app.run_server(debug=True)



In [ ]:
# import pandas as pd
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# # Example tickers list

# df.index = df.index.set_levels([df.index.levels[0], pd.to_datetime(df.index.levels[1])])

# # Define the cutoff date
# cutoff_date = pd.to_datetime('2023-01-01')

# models = {}
# mean_sqr_error = {}
# predictions_dict = {}
# actuals_dict = {}
# dates_dict = {}

# count = 0
# for tick in tickers:
#     stock_data = df.loc[tick].copy() 

    
#     # Prepare features and target
#     X = stock_data[['lag1', 'lag2', 'lag3', 'rolling_mean', 'rolling_std', 'volume', 'rsi', 'bb_low', 'bb_mid', 'bb_high', 'atr', 'macd', 'macd_signal', 'return']]
#     y = stock_data['adj close']

#     # Split into training and testing sets
#     train_data = stock_data.loc[stock_data.index < stock_data.index[round(len(stock_data.index) * .872)]]
#     test_data = stock_data.loc[stock_data.index >= stock_data.index[round(len(stock_data.index) * .872)]]
    
#     X_train = train_data.drop(columns=['adj close'])
#     y_train = train_data['adj close']
#     X_test = test_data.drop(columns=['adj close'])
#     y_test = test_data['adj close']
    
#     # Model Training
#     model = RandomForestRegressor(n_estimators=100, random_state=42)
    
#     print(f'Shape: {X_train.shape}')
#     model.fit(X_train, y_train)
    
#     # Save the model
#     models[tick] = model
    
#     # Model Prediction and Evaluation
#     predictions = model.predict(X_test)
    
#     predictions_dict[tick] = predictions
#     actuals_dict[tick] = y_test.values
#     dates_dict[tick] = test_data.index
    
#     mean_sqr_error[tick] = mean_squared_error(y_test, predictions)
#     mse = mean_sqr_error[tick]
    
#     count +=1 
#     print(f'{count}: {tick} - Mean Squared Error: {mse}')
    



In [ ]:
# import pickle as pk

# with open('MSE.pkl', 'wb') as f:
#     pk.dump(mean_sqr_error, f)
    
# with open('dates.pkl', 'wb') as f:
#     pk.dump(dates_dict, f)
    
# with open('actuals.pkl', 'wb') as f:
#     pk.dump(actuals_dict, f)
    
# with open('predictions.pkl', 'wb') as f:
#     pk.dump(predictions_dict, f)

In [15]:
import pickle as pk

with open('Data/MSE.pkl', 'rb') as f:
    mean_sqr_error = pk.load(f)
with open ('Data/dates.pkl', 'rb') as f:
    dates_dict = pk.load(f)
with open ('Data/actuals.pkl', 'rb') as f:
    actuals_dict = pk.load(f)
with open ('Data/predictions.pkl', 'rb') as f:
    predictions_dict = pk.load(f)

In [17]:
with open('Data/stock_model_2.pkl', 'rb') as f:
    models = pk.load(f)

In [18]:
predictions_dict

{'A': array([149.51239975, 149.38645004, 145.28622787, 145.34387848,
        153.08710129, 155.57070084, 153.93051315, 155.12582672,
        153.57847321, 151.77189423, 151.09893997, 153.66656845,
        156.32788452, 153.54671371, 153.60256439, 153.80138672,
        153.49808075, 149.6069194 , 149.47188171, 153.60869278,
        153.53007294, 152.10819778, 150.04743393, 152.00499512,
        151.77054367, 150.07250702, 150.1033989 , 152.03687149,
        150.85093079, 149.39278046, 146.45907028, 146.40737244,
        141.20269638, 140.1813208 , 140.26119614, 138.88508774,
        140.18518951, 139.2885141 , 135.6443364 , 138.89340256,
        141.54453003, 140.95807495, 138.72976761, 138.93280807,
        137.46030304, 133.82522507, 134.47092056, 136.57446609,
        132.18407837, 134.84539093, 131.28240768, 132.7045871 ,
        135.10959137, 132.00374725, 129.32379578, 130.1412558 ,
        131.19042328, 131.13202377, 131.9859642 , 132.60407272,
        136.40097412, 135.88598587,

In [19]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd


# Create the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Stock Prediction Model Comparison"),
    dcc.Dropdown(
        id='ticker-dropdown',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value=tickers[0],  # default value
        clearable=False
    ),
    dcc.Graph(id='prediction-graph')
])

# Callback to update the graph
@app.callback(
    Output('prediction-graph', 'figure'),
    [Input('ticker-dropdown', 'value')]
)
def update_graph(selected_ticker):
    predictions = predictions_dict[selected_ticker]
    actuals = actuals_dict[selected_ticker]
    dates = dates_dict[selected_ticker]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates,
        y=actuals,
        mode='lines',
        name='Actual'
    ))

    fig.add_trace(go.Scatter(
        x=dates,
        y=predictions,
        mode='lines',
        name='Predicted'
    ))

    fig.update_layout(
        title=f'Actual vs Predicted values for {selected_ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [63]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from ipywidgets import interact, widgets

def plot_stock(ticker):
    if ticker not in tickers:
        return
    
    predictions = predictions_dict[ticker]
    actuals = actuals_dict[ticker]
    dates = dates_dict[ticker]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates,
        y=actuals,
        mode='lines',
        name='Actual',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=dates,
        y=predictions,
        mode='lines',
        name='Predicted',
        line=dict(color='red')
    ))

    fig.update_layout(
        title=f'Actual vs Predicted values for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )
    
    return fig.show()

ticker_dropdown = widgets.Dropdown(
    options=tickers,
    value=None, 
    description='Ticker:',
)

def update_plot(ticker):
    if ticker:
        plot_stock(ticker)

interact(update_plot, ticker=ticker_dropdown)


interactive(children=(Dropdown(description='Ticker:', options=('A', 'AAL', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACG…

<function __main__.update_plot(ticker)>

In [69]:
import plotly.io as pio

figures_json = {}
for ticker in tickers:
    predictions = predictions_dict[ticker]
    actuals = actuals_dict[ticker]
    dates = dates_dict[ticker]
    
    # Retrieve the Mean Squared Error from the dictionary
    mse = mean_sqr_error[ticker]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates,
        y=actuals,
        mode='lines',
        name='Actual',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=dates,
        y=predictions,
        mode='lines',
        name=f'Predicted (MSE: {mse:.2f})',
        line=dict(color='red')
    ))

    fig.update_layout(
        title=f'Actual vs Predicted values for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )
    
    figures_json[ticker] = pio.to_json(fig)

# Create a single HTML file with a dropdown and embedded Plotly graphs
html_content = '''
<!DOCTYPE html>
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1>Stock Price Predictions</h1>
    <label for="ticker-select">Choose a ticker:</label>
    <select id="ticker-select" onchange="updatePlot()">
        {options}
    </select>
    <div id="plot"></div>
    <script>
        var figures = {figures};
        
        function updatePlot() {{
            var ticker = document.getElementById('ticker-select').value;
            var graphDiv = document.getElementById('plot');
            Plotly.react(graphDiv, JSON.parse(figures[ticker]).data, JSON.parse(figures[ticker]).layout);
        }}
        
        // Initial plot
        document.addEventListener('DOMContentLoaded', function() {{
            updatePlot();
        }});
    </script>
</body>
</html>
'''

options = ''.join([f'<option value="{ticker}">{ticker}</option>' for ticker in tickers])
html_content = html_content.format(options=options, figures=figures_json)

# Save the HTML content to a file
with open('stock_predictions.html', 'w') as f:
    f.write(html_content)

print("HTML file with dropdown has been generated.")

HTML file with dropdown has been generated.
